In [ ]:


import pandas as pd
from pathlib import Path
import re

def analyze_experiment_results(env_name, filter_name=[None]):
    # Base path for the logs
    base_path = Path('../logs') / env_name / 'eval'
    
    # Store results
    results = []
    
    # Find all experiment directories
    for exp_dir in base_path.glob('*'):
        if not exp_dir.is_dir():
            continue
            
        results_file = exp_dir / 'results.txt'
        if not results_file.exists():
            continue
            
        # Read and parse results
        with open(results_file, 'r') as f:
            content = f.read()
            
        # Extract score and std using regex
        match = re.search(r'score: ([\d.]+) \+\- ([\d.]+)', content)
        if match:
            score = float(match.group(1))
            std = float(match.group(2))
            
            not_match = False
            for n in filter_name:
                if n is not None and n not in exp_dir.name:
                    not_match = True
                    break
            if not_match:
                continue
            
            results.append({
                'experiment': exp_dir.name,
                'score': score,
                'std': std
            })
    
    # Create DataFrame and sort by score
    df = pd.DataFrame(results)
    if not df.empty:
        df = df.sort_values('score', ascending=False)
    
    return df

# Example usage:
env_name = 'hopper-medium-replay-v2'
results_df = analyze_experiment_results(env_name, filter_name=['mc', 'ot'])
print(f"\nResults for {env_name}:")
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None) 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping

print(results_df)